In [ ]:
import cv2 
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
from pandas import *
import numpy as np
import math
import time

capture = cv2.VideoCapture(0)
detect= HandDetector(maxHands=1)
classifier = Classifier("Model/keras_model.h5","Model/labels.txt")


data = read_csv("Model/labels.csv")
labels= data['value'].tolist()
offset=20
size=300

while True:
    success, img = capture.read()
    final_Output= img.copy()
    hands, img = detect.findHands(img)
    
    if hands:
        hands=hands[0]
        x,y,w,h=hands['bbox']

        cropppedImg= img[y- offset:y + h + offset, x-offset:x + w+offset]
        imgWhite= np.ones((size,size,3),np.uint8)*255

        cropppedImgShape= cropppedImg.shape


        aspectRatio=h/w

        if aspectRatio >1 :
            k= size/h
            wCal= math.ceil(k*w)
            imgResize= cv2.resize(cropppedImg,(wCal,size))
            imgResizeShape= imgResize.shape
            wGap = math.ceil((size - wCal) / 2)
            imgWhite[:, wGap:wCal+wGap] = imgResize
            prediction,index= classifier.getPrediction(imgWhite,draw=False)
            print(prediction,index)

        else:
            k= size/w
            hCal=math.ceil(k*h)
            imgResize= cv2.resize(cropppedImg,(size,hCal))
            imgResizeShape= imgResize.shape
            hGap=math.ceil((size - hCal) / 2)
            imgWhite[hGap:hCal+hGap, :] = imgResize
            prediction,index= classifier.getPrediction(imgWhite,draw=False)

        
        cv2.putText(final_Output,labels[index],(x,y-20), cv2.FONT_HERSHEY_DUPLEX,2,(240,248,255),4)
        cv2.imshow("ImageCrop",cropppedImg)
        cv2.imshow("ImageWhite",imgWhite)
    
    
    cv2.imshow("Image",final_Output)
    key = cv2.waitKey(1)

